# Generating images with MX-Font model from a reference style
In this example we'll generate images with trained MX-Font model from a reference style.
If you want to generate multiple styles, please check using `eval.py` instead of using this example file (because it is much simpler to load the referece styles).

### 1. Loading packages
* First, load the packages used in this code.
* All of the packages are avilable in `pip`.

In [1]:
import json
from pathlib import Path
from PIL import Image

import torch
from sconf import Config
from torchvision import transforms

* These modules are defined in this repository.

In [2]:
import models
from datasets import read_font, render
from utils import save_tensor_to_image

### 2. Build model
* Build and load the trained model.
* `weight_path` : 
    * The location of the trained model weight.

In [3]:
########################################################
weight_path = "result/checkpoints-4200/26000.pth"  # path to weight to infer
########################################################

cfg = Config("cfgs/eval.yaml", default="cfgs/defaults.yaml")
transform = transforms.Compose(
    [transforms.Resize((128, 128)), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
)
decomposition = json.load(open("data/chn_decomposition.json"))

g_kwargs = cfg.get('g_args', {})
gen = models.Generator(1, cfg.C, 1, **g_kwargs).cuda().eval()
weight = torch.load(weight_path)
if "generator_ema" in weight:
    weight = weight["generator_ema"]
gen.load_state_dict(weight)

<All keys matched successfully>

### 3. Load reference images.
* `ref_path`: 
    * The path of reference font or images.
    * If you are using a ttf file, set this to the location of the ttf file.
    * If you want to use rendered images, set this to the path to the directory which contains the reference images.
* `extension`:
    * If you are using image files, set this to their extension(png, jpg, etc..). 
    * This will be ignored if `use_ttf` is True.
* `batch_size`:
    * The number of images inferred at once.

In [4]:
########################################################
ref_path = "data/images/test/MaShanZheng-Regular/"  # Path to the reference images
extension = "png"  # Extension of the reference images
batch_size = 3  # The batch size
########################################################

ref_paths = Path(ref_path).glob(f"*.{extension}")
ref_imgs = torch.stack([transform(Image.open(str(p))) for p in ref_paths]).cuda()
ref_batches = torch.split(ref_imgs, batch_size)

### 4. Extract style factors from reference images.

In [5]:
style_facts = {}

for batch in ref_batches:
    style_fact = gen.factorize(gen.encode(batch), 0)
    for k in style_fact:
        style_facts.setdefault(k, []).append(style_fact[k])
        
style_facts = {k: torch.cat(v).mean(0, keepdim=True) for k, v in style_facts.items()}

### 5. Generate the images.
* `gen_chars`: The characters to generate.
* `save_dir`: Path to save the generated images.
* `source_path`: Path to the font file used as the source font.

In [6]:
########################################################
gen_chars = "頒騎亀願嵐紡飲鄭嬰鯉従鐙勢読梼殻掃緊蘇勧墳図"  # Characters to generate
save_dir = Path("./result_test")  # Directory where you want to save generated images
source_path = "data/ttfs/test/ipag.ttf"  # Path to the font file to render the source images
########################################################

save_dir.mkdir(parents=True, exist_ok=True)

source_font = read_font(source_path)
for char in gen_chars:
    source_img = transform(render(source_font, char)).unsqueeze(0).cuda()
    char_facts = gen.factorize(gen.encode(source_img), 1)
    
    gen_feats = gen.defactorize([style_facts, char_facts])
    out = gen.decode(gen_feats).detach().cpu()[0]

    path = save_dir / f"{char}.png"
    save_tensor_to_image(out, path)

In [7]:
char_facts

{'last': tensor([[[[[-8.5233e-01,  1.3234e+00, -8.4224e-01,  ...,  9.3823e-01,
             -1.4890e+00, -2.6197e+00],
            [-1.2873e+00,  1.6011e-01, -2.6120e-01,  ...,  9.3617e-01,
             -2.0043e+00, -2.8959e+00],
            [-2.3687e+00,  4.2545e-02,  6.7763e-01,  ..., -8.7998e-01,
             -2.1673e+00, -2.2859e+00],
            ...,
            [-2.4487e-01, -1.6179e-01, -7.4074e+00,  ..., -1.7547e+00,
             -2.0713e+00, -3.4736e+00],
            [-1.1098e+00,  4.5638e+00, -6.4255e-01,  ...,  1.3827e+00,
             -4.9703e-01, -2.5231e+00],
            [ 9.4169e-01,  5.5803e+00,  1.8349e+00,  ...,  1.6984e+00,
              1.8986e+00,  4.1788e-01]],
 
           [[-1.5818e+00,  3.6950e-01, -3.5141e-01,  ...,  5.9532e-01,
             -2.7948e+00, -1.7478e+00],
            [-2.7525e+00,  2.3060e+00,  2.5255e+00,  ...,  4.2479e+00,
              1.5675e-01, -1.0668e+00],
            [-4.5497e+00,  1.4498e+00,  1.6510e+00,  ...,  4.3062e+00,
             